## CSE 158 HW4
### Christina Leung, A15468909

In [1]:
from scipy.sparse import lil_matrix
from nltk.corpus import stopwords
import gzip
from collections import defaultdict
from collections import Counter 
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
from sklearn import linear_model
import nltk
from nltk.stem.porter import *

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
data = []
for d in readGz("train_Category.json.gz"):
    data.append(d)

In [3]:
firstten = []
for i in range(0, 10000):
    firstten.append(data[i]['text'].translate(str.maketrans('', '', string.punctuation)).lower())

In [4]:
# QUESTION 1
counts = defaultdict(int)
for l in firstten:
    b = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for val in b:
        counts[val] += 1
counts = [(counts[w], w) for w in counts]
counts.sort()
counts.reverse()
grams = [x for x in counts[:5]]
print(grams)
print(len(counts))

[(4092, ('this', 'game')), (3867, ('the', 'game')), (3355, ('of', 'the')), (1988, ('in', 'the')), (1910, ('game', 'is'))]
266396


## Question 1
There are 266396 unique bigrams. The five most popular ones are (format is (count, bigram):<br>
[(4092, ('this', 'game')), (3867, ('the', 'game')), (3355, ('of', 'the')), (1988, ('in', 'the')), (1910, ('game', 'is'))]

In [5]:
# QUESTION 2
corpus = [x[1] for x in counts[:1000]]
gramId = dict(zip(corpus, range(len(corpus))))
gramSet = set(corpus)

def feature(datum):
    feat = [0]*len(gramSet)
    l = datum
    r = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for w in r:
        if w in gramSet:
            feat[gramId[w]] += 1
    feat.append(1)
    return feat
def res(datum):
    return math.log(datum['hours'] + 1, 2)
    
X = [feature(d) for d in firstten]
y = [res(d) for d in data[0:10000]]

clf = linear_model.Ridge(1.0, fit_intercept=False) 
clf.fit(X, y)

In [6]:
est = clf.predict(X)
mse = numpy.square(numpy.subtract(est, y)).mean()
print(mse)

4.409616426571927


## Question 2
MSE = 4.409616426571927

In [7]:
# QUESTION 3
counts = defaultdict(int)
for l in firstten:
    b = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for val in b:
        counts[val] += 1
    b = l.split(" ")
    for val in b:
        counts[(val, '')] += 1
counts = [(counts[w], w) for w in counts]
counts.sort()
counts.reverse()

In [8]:
corpus = [x[1] for x in counts[:1000]]
gramId = dict(zip(corpus, range(len(corpus))))
gramSet = set(corpus)
def feature2(datum):
    feat = [0]*len(gramSet)
    l = datum
    r = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for w in r:
        if w in gramSet:
            feat[gramId[w]] += 1
    r = l.split(" ")
    for w in r:
        if (w,'') in gramSet:
            feat[gramId[(w,'')]] += 1
    feat.append(1)
    return feat
    
X2 = [feature2(d) for d in firstten]
y2 = [res(d) for d in data[0:10000]]

clf = linear_model.Ridge(1.0, fit_intercept=False) 
clf.fit(X2, y2)

Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [9]:
est = clf.predict(X2)
mse = numpy.square(numpy.subtract(est, y2)).mean()
print(mse)

4.2428508756634855


## Question 3
MSE = 4.2428508756634855

In [10]:
# QUESTION 4
df = defaultdict(int)
unigrams = defaultdict(int)
for d in firstten:
    curr = set([])
    tokens = d.split()
    for w in tokens:
        unigrams[w] += 1
        if w not in curr:
            curr.add(w)
            df[w] += 1
ugrams = [(unigrams[w], w) for w in unigrams]
ugrams.sort()
ugrams.reverse()

In [11]:
print('Inverse Document Frequencies:')
tmp1 = math.log10(len(firstten)/(1 + df['destiny']))
print('destiny:')
print(tmp1)
tmp2 = math.log10(len(firstten)/(1 + df['annoying']))
print('annoying:')
print(tmp2)
tmp3 = math.log10(len(firstten)/(1 + df['likeable']))
print('likeable')
print(tmp3)
tmp4 = math.log10(len(firstten)/(1 + df['chapter']))
print('chapter')
print(tmp4)
tmp5 = math.log10(len(firstten)/(1 + df['interesting']))
print('interesting')
print(tmp5)

Inverse Document Frequencies:
destiny:
3.3010299956639813
annoying:
1.8356471442155629
likeable
3.0457574905606752
chapter
2.214670164989233
interesting
1.3575354797578787


In [11]:
wordtime = defaultdict(int)
ofinterest = None
for d in data:
    if d['reviewID'] == 'r75487422':
        ofinterest = d
        rt = d['text'].translate(str.maketrans('', '', string.punctuation)).lower().split(' ')
        for w in rt:
            wordtime[w] += 1
        break

In [13]:
print('TF-IDF:')
tmp1 = tmp1 * wordtime['destiny']
print('destiny:')
print(tmp1)
tmp2 = tmp2 * wordtime['annoying']
print('annoying:')
print(tmp2)
tmp3 = tmp3 * wordtime['likeable']
print('likeable')
print(tmp3)
tmp4 = tmp4 * wordtime['chapter']
print('chapter')
print(tmp4)
tmp5 = tmp5 * wordtime['interesting']
print('interesting')
print(tmp5)

TF-IDF:
destiny:
3.3010299956639813
annoying:
3.6712942884311257
likeable
6.0915149811213505
chapter
6.644010494967699
interesting
2.7150709595157574


## Question 4
Inverse Document Frequencies:<br>
destiny:<br>
3.3010299956639813<br>
annoying:<br>
1.8356471442155629<br>
likeable<br>
3.0457574905606752<br>
chapter<br>
2.214670164989233<br>
interesting<br>
1.3575354797578787<br><br>

TF-IDF:<br>
destiny:<br>
3.3010299956639813<br>
annoying:<br>
3.6712942884311257<br>
likeable<br>
6.0915149811213505<br>
chapter<br>
6.644010494967699<br>
interesting<br>
2.7150709595157574<br>

In [12]:
# QUESTION 5
ugrams = [x[1] for x in ugrams[:1000]]
gramId = dict(zip(ugrams, range(len(ugrams))))
gramSet = set(ugrams)
def feature3(datum):
    feat = [0]*len(gramSet)
    r = datum.split(' ')
    currcount = defaultdict(int)
    for word in r:
        currcount[word] += 1
    myset = set([])
    for word in r:
        if word in gramSet and not word in myset:
            myset.add(word)
            num = currcount[word]
            denom = math.log10(len(firstten)/(1 + df[word]))
            feat[gramId[word]] = (num * denom)
    feat.append(1)
    return feat
    
X2 = [feature3(d) for d in firstten]
y2 = [res(d) for d in data[0:10000]]

clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X2, y2)

Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [13]:
est = clf.predict(X2)
mse = numpy.square(numpy.subtract(est, y2)).mean()
print(mse)

4.2135836747754345


## Question 5
MSE = 4.2135836747754345

In [14]:
# QUESTION 6
arr1 = [0] * len(gramSet)
text1 = ofinterest['text'].translate(str.maketrans('', '', string.punctuation)).lower().split()
currcount = defaultdict(int)
for val in text1:
    currcount[val] += 1
for val in text1:
    if val in gramSet:
        p1 = currcount[val]
        p2 = math.log10(len(gramSet) / (1 + df[val]))
        arr1[gramId[val]] = (p1 * p2)

In [15]:
mmax = 0
rmax = ''
for i in range(0, 10000):
    arr2 = [0] * len(gramSet)
    curr = data[i]['text'].translate(str.maketrans('', '', string.punctuation)).lower().split()
    currcount = defaultdict(int)
    for val in curr:
        currcount[val] += 1
    for val in curr:
        if val in gramSet:
            p1 = currcount[val]
            p2 = math.log10(len(gramSet) / (1 + df[val]))
            arr2[gramId[val]] = (p1 * p2)
    t1 = sum(i[0] * i[1] for i in zip(arr1, arr2))
    t2 = math.sqrt(sum(numpy.array(arr1)**2)) + math.sqrt(sum(numpy.array(arr2)**2))
    sim = t1/t2
    if sim > mmax:
        mmax = sim
        rmax = data[i]['reviewID']
print(rmax)

r33215456


## Question 6
most similar reviewId: r33215456

In [46]:
# QUESTION 7
# train_X.append(data[i]['text'].translate(str.maketrans('', '', string.punctuation)).lower())
random.shuffle(data)
test_X = []
train_X = []
valid_X = []
test_y = []
train_y = []
valid_y = []
for i in range(0, len(data)):
    if i < 10000:
        train_y.append(math.log(data[i]['hours'] + 1, 2))
        train_X.append(data[i]['text'])
    elif i < 20000:
        test_y.append(math.log(data[i]['hours'] + 1, 2))
        test_X.append(data[i]['text'])
    elif i < 30000:
        valid_y.append(math.log(data[i]['hours'] + 1, 2))
        valid_X.append(data[i]['text'])

In [47]:
# Unigrams, removing punctuation, tfidf scores
counts = defaultdict(int)
unpdf = defaultdict(int)
for l in train_X:
    curr = set([])
    l = l.translate(str.maketrans('', '', string.punctuation)).lower()
    b = l.split(" ")
    for val in b:
        counts[val] += 1
        if val not in curr:
            curr.add(val)
            unpdf[val] += 1
counts = [(counts[w], w) for w in counts]
counts.sort()
counts.reverse()
unp = [x[1] for x in counts[:1000]]
unpId = dict(zip(unp, range(len(unp))))
unpSet = set(unp)

In [48]:
def feature10(datum):
    feat = [0]*len(unpSet)
    r = datum.translate(str.maketrans('', '', string.punctuation)).lower().split(' ')
    currcount = defaultdict(int)
    for word in r:
        currcount[word] += 1
    myset = set([])
    for word in r:
        if word in unpSet and not word in myset:
            myset.add(word)
            p1 = currcount[word]
            p2 = math.log10(len(train_X)/(1 + unpdf[word]))
            feat[unpId[word]] = (p1 * p2)
    feat.append(1)
    return feat
    
X = [feature10(d) for d in train_X]
t3 = [feature10(d) for d in test_X]
t2 = [feature10(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[4.448111788052944, 6.761583602001666, 6.85650805314973]
[4.448111836548716, 6.76097112404808, 6.855889569648352]
[4.448116643001447, 6.754887628743253, 6.849747451429933]
[4.44855778688827, 6.697890672530793, 6.792289950852203]
[4.4720878881908215, 6.349980586845211, 6.44413898123869]


In [49]:
# Unigrams, removing punctuation, word counts
def feature11(datum):
    feat = [0]*len(unpSet)
    r = datum.translate(str.maketrans('', '', string.punctuation)).lower().split(' ')
    for word in r:
        if word in unpSet:
            feat[unpId[word]] += 1
    feat.append(1)
    return feat
    
X = [feature11(d) for d in train_X]
t3 = [feature11(d) for d in test_X]
t2 = [feature11(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[4.448111794891627, 6.761403279373198, 6.856325280901804]
[4.448112517466186, 6.759173890482406, 6.854068047784618]
[4.448181900249314, 6.737494585242364, 6.8321313363385645]
[4.45307728156437, 6.567159742241056, 6.66062495819836]
[4.547305386666309, 6.041192110869887, 6.139042048101774]


In [50]:
# Unigrams, keeping punctuation, tfidf scores
counts = defaultdict(int)
updf = defaultdict(int)
for l in train_X:
    b = l.lower().split(" ")
    curr = set([])
    for val in b:
        counts[val] += 1
        if val not in curr:
            curr.add(val)
            updf[val] += 1
counts = [(counts[w], w) for w in counts]
counts.sort()
counts.reverse()
up = [x[1] for x in counts[:1000]]
upId = dict(zip(up, range(len(up))))
upSet = set(up)

In [51]:
def feature12(datum):
    feat = [0]*len(upSet)
    r = datum.lower().split(' ')
    currcount = defaultdict(int)
    for word in r:
        currcount[word] += 1
    myset = set([])
    for word in r:
        if word in upSet and not word in myset:
            myset.add(word)
            p1 = currcount[word]
            p2 = math.log10(len(train_X)/(1 + updf[word]))
            feat[upId[word]] = (p1 * p2)
    feat.append(1)
    return feat
    
X = [feature12(d) for d in train_X]
t3 = [feature12(d) for d in test_X]
t2 = [feature12(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[4.483481799621346, 8.055930993379272, 6.929421781910879]
[4.483481854798076, 8.05478999731367, 6.928746967050129]
[4.483487320844511, 8.043433187985066, 6.922042850334407]
[4.483986836779045, 7.934931964234332, 6.859096456060031]
[4.509951965909808, 7.1959706553821325, 6.468977918645232]


In [52]:
# Unigrams, keeping punctuation, word counts
def feature13(datum):
    feat = [0]*len(upSet)
    r = datum.lower().split(' ')
    for word in r:
        if word in upSet:
            feat[upId[word]] += 1
    feat.append(1)
    return feat
    
X = [feature13(d) for d in train_X]
t3 = [feature13(d) for d in test_X]
t2 = [feature13(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[4.483481807767196, 8.055482055923203, 6.9292033575724465]
[4.4834826656453775, 8.050309109877, 6.926569584066258]
[4.483564826768958, 7.999460653274094, 6.900932228158309]
[4.489260782117701, 7.567906247692666, 6.69772907896097]
[4.594636508523769, 6.161785145786728, 6.077082820205692]


In [71]:
# Bigrams, removing punctuation, tfidf scores
counts = defaultdict(int)
bnpdf = defaultdict(int)
for l in train_X:
    curr = set([])
    l = l.translate(str.maketrans('', '', string.punctuation)).lower()
    b = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for val in b:
        counts[val] += 1
        if val not in curr:
            curr.add(val)
            bnpdf[val] += 1
counts = [(counts[w], w) for w in counts]
counts.sort()
counts.reverse()
bnp = [x[1] for x in counts[:1000]]
bnpId = dict(zip(bnp, range(len(bnp))))
bnpSet = set(bnp)

In [64]:
def feature14(datum):
    feat = [0]*len(bnpSet)
    r = datum.translate(str.maketrans('', '', string.punctuation)).lower()
    r = zip(r.split(" ")[:-1], r.split(" ")[1:])
    currcount = defaultdict(int)
    for word in r:
        currcount[word] += 1
    myset = set([])
    for word in r:
        if word in bnpSet and not word in myset:
            myset.add(word)
            p1 = currcount[word]
            p2 = math.log10(len(train_X)/(1 + bnpdf[word]))
            feat[bnpId[word]] = (p1 * p2)
    feat.append(1)
    return feat
    
X = [feature14(d) for d in train_X]
t3 = [feature14(d) for d in test_X]
t2 = [feature14(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[5.382084413516019, 5.264208278873948, 5.350977452054177]
[5.382084414853578, 5.264204046223508, 5.350975803183231]
[5.382084548585164, 5.264161844266324, 5.350959436463311]
[5.382097897488407, 5.263752254986487, 5.350807944042781]
[5.383408888294626, 5.260875320278863, 5.350486679325185]


In [65]:
# Bigrams, removing punctuation, word counts
def feature15(datum):
    feat = [0]*len(bnpSet)
    l = datum.translate(str.maketrans('', '', string.punctuation)).lower()
    r = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for w in r:
        if w in bnpSet:
            feat[bnpId[w]] += 1
    feat.append(1)
    return feat
    
X = [feature15(d) for d in train_X]
t3 = [feature15(d) for d in test_X]
t2 = [feature15(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[4.576831630473613, 6.73516330313452, 6.520377392365218]
[4.576849378370181, 6.730637016758092, 6.515133150076188]
[4.577563347221398, 6.692133363632492, 6.47598060761095]
[4.591030736205471, 6.4242754638905115, 6.2522368583275005]
[4.74948617532988, 5.729936754893262, 5.742410956602475]


In [70]:
# Bigrams, keeping punctuation, tfidf scores
counts = defaultdict(int)
bpdf = defaultdict(int)
for l in train_X:
    curr = set([])
    l = l.lower()
    b = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for val in b:
        counts[val] += 1
        if val not in curr:
            curr.add(val)
            bpdf[val] += 1
counts = [(counts[w], w) for w in counts]
counts.sort()
counts.reverse()
bp = [x[1] for x in counts[:1000]]
bpId = dict(zip(bp, range(len(bp))))
bpSet = set(bp)

In [67]:
def feature16(datum):
    feat = [0]*len(bpSet)
    r = datum.lower()
    r = zip(r.split(" ")[:-1], r.split(" ")[1:])
    currcount = defaultdict(int)
    for word in r:
        currcount[word] += 1
    myset = set([])
    for word in r:
        if word in bpSet and not word in myset:
            myset.add(word)
            p1 = currcount[word]
            p2 = math.log10(len(train_X)/(1 + bpdf[word]))
            feat[bpId[word]] = (p1 * p2)
    feat.append(1)
    return feat
    
X = [feature16(d) for d in train_X]
t3 = [feature16(d) for d in test_X]
t2 = [feature16(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[5.382084413516019, 5.264208278873948, 5.350977452054177]
[5.382084414853578, 5.264204046223508, 5.350975803183231]
[5.382084548585164, 5.264161844266324, 5.350959436463311]
[5.382097897488407, 5.263752254986487, 5.350807944042781]
[5.383408888294626, 5.260875320278863, 5.350486679325185]


In [68]:
# Bigrams, keeping punctuation, word counts
def feature17(datum):
    feat = [0]*len(bpSet)
    l = datum.lower()
    r = zip(l.split(" ")[:-1], l.split(" ")[1:])
    for w in r:
        if w in bpSet:
            feat[bpId[w]] += 1
    feat.append(1)
    return feat
    
X = [feature17(d) for d in train_X]
t3 = [feature17(d) for d in test_X]
t2 = [feature17(d) for d in valid_X]

c1 = []
c2 = []
c3 = []
c4 = []
c5 = []

clf = linear_model.Ridge(0.01, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c1.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c1.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c1.append(mse)

clf = linear_model.Ridge(0.1, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c2.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c2.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c2.append(mse)

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c3.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c3.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c3.append(mse)

clf = linear_model.Ridge(10, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c4.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c4.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c4.append(mse)

clf = linear_model.Ridge(100, fit_intercept=False)
clf.fit(X, train_y)
pred = clf.predict(X)
mse = numpy.square(numpy.subtract(pred, train_y)).mean()
c5.append(mse)
pred = clf.predict(t2)
mse = numpy.square(numpy.subtract(pred, test_y)).mean()
c5.append(mse)
pred = clf.predict(t3)
mse = numpy.square(numpy.subtract(pred, valid_y)).mean()
c5.append(mse)

print(c1)
print(c2)
print(c3)
print(c4)
print(c5)

[4.61928318160182, 6.396641142375481, 6.450978816279689]
[4.619297123874244, 6.3919217294605835, 6.446146919080924]
[4.620013766561267, 6.352872293303977, 6.407180943004867]
[4.635516874738719, 6.1220628734252776, 6.1798517883637105]
[4.800472344835454, 5.622612489935406, 5.693520746413202]


## Question 7
Format is [training MSE, validation MSE, testing MSE]<br>
Model MSEs are listed in order of C = 0.01, 0.1, 1.0, 10.0, 100.0<br><br>
Unigrams, removing punctuation, tfidf scores<br>
[4.448111788052944, 6.761583602001666, 6.85650805314973]<br>
[4.448111836548716, 6.76097112404808, 6.855889569648352]<br>
[4.448116643001447, 6.754887628743253, 6.849747451429933]<br>
[4.44855778688827, 6.697890672530793, 6.792289950852203]<br>
[4.4720878881908215, 6.349980586845211, 6.44413898123869]<br><br>

Unigrams, removing punctuation, word counts<br>
[4.448111794891627, 6.761403279373198, 6.856325280901804]<br>
[4.448112517466186, 6.759173890482406, 6.854068047784618]<br>
[4.448181900249314, 6.737494585242364, 6.8321313363385645]<br>
[4.45307728156437, 6.567159742241056, 6.66062495819836]<br>
[4.547305386666309, 6.041192110869887, 6.139042048101774]<br><br>

Unigrams, keeping punctuation, tfidf scores<br>
[4.483481799621346, 8.055930993379272, 6.929421781910879]<br>
[4.483481854798076, 8.05478999731367, 6.928746967050129]<br>
[4.483487320844511, 8.043433187985066, 6.922042850334407]<br>
[4.483986836779045, 7.934931964234332, 6.859096456060031]<br>
[4.509951965909808, 7.1959706553821325, 6.468977918645232]<br><br>

Unigrams, keeping punctuation, word counts<br>
[4.483481807767196, 8.055482055923203, 6.9292033575724465]<br>
[4.4834826656453775, 8.050309109877, 6.926569584066258]<br>
[4.483564826768958, 7.999460653274094, 6.900932228158309]<br>
[4.489260782117701, 7.567906247692666, 6.69772907896097]<br>
[4.594636508523769, 6.161785145786728, 6.077082820205692]<br><br>

Bigrams, removing punctuation, tfidf scores<br>
[5.382084413516019, 5.264208278873948, 5.350977452054177]<br>
[5.382084414853578, 5.264204046223508, 5.350975803183231]<br>
[5.382084548585164, 5.264161844266324, 5.350959436463311]<br>
[5.382097897488407, 5.263752254986487, 5.350807944042781]<br>
[5.383408888294626, 5.260875320278863, 5.350486679325185]<br><br>

Bigrams, removing punctuation, word counts<br>
[4.576831630473613, 6.73516330313452, 6.520377392365218]<br>
[4.576849378370181, 6.730637016758092, 6.515133150076188]<br>
[4.577563347221398, 6.692133363632492, 6.47598060761095]<br>
[4.591030736205471, 6.4242754638905115, 6.2522368583275005]<br>
[4.74948617532988, 5.729936754893262, 5.742410956602475]<br><br>

Bigrams, keeping punctuation, tfidf scores<br>
[5.382084413516019, 5.264208278873948, 5.350977452054177]<br>
[5.382084414853578, 5.264204046223508, 5.350975803183231]<br>
[5.382084548585164, 5.264161844266324, 5.350959436463311]<br>
[5.382097897488407, 5.263752254986487, 5.350807944042781]<br>
[5.383408888294626, 5.260875320278863, 5.350486679325185]<br><br>

Bigrams, keeping punctuation, word counts<br>
[4.61928318160182, 6.396641142375481, 6.450978816279689]<br>
[4.619297123874244, 6.3919217294605835, 6.446146919080924]<br>
[4.620013766561267, 6.352872293303977, 6.407180943004867]<br>
[4.635516874738719, 6.1220628734252776, 6.1798517883637105]<br>
[4.800472344835454, 5.622612489935406, 5.693520746413202]<br><br>

Best validation set performance is bigrams, tfidf scores, (removing puncutation or keeping punctuation), and C = 100. The validation MSE is 5.260875320278863 and the test MSE is 5.350486679325185 for both models.